In [1]:
#importing the required libraries
import numpy as np
import pandas as pd
import pickle
#import matrix_factorization_utilities
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.neighbors import NearestNeighbors


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
movie_wide=pd.read_csv('project12/one.csv')

In [3]:
#reading the movies dataset
movie_list = pd.read_csv('old dataset/movies.csv')

In [6]:
tags = pd.read_csv('old dataset/tags.csv')

In [7]:
# inspecting various genres
genres = movie_list['genres']

In [8]:
genre_list = ""
for index,row in movie_list.iterrows():
        genre_list += row.genres + "|"
#split the string into a list of values
genre_list_split = genre_list.split('|')
#de-duplicate values
new_list = list(set(genre_list_split))
#remove the value that is blank
new_list.remove('')
#inspect list of genres
new_list

['IMAX',
 'Western',
 'Children',
 'Drama',
 'Adventure',
 'Film-Noir',
 'Thriller',
 'Mystery',
 'Fantasy',
 'Comedy',
 'Crime',
 'Animation',
 'Action',
 'Horror',
 'War',
 '(no genres listed)',
 'Romance',
 'Musical',
 'Documentary',
 'Sci-Fi']

In [9]:
#Enriching the movies dataset by adding the various genres columns.
movies_with_genres = movie_list.copy()

for genre in new_list :
    movies_with_genres[genre] = movies_with_genres.apply(lambda _:int(genre in _.genres), axis = 1)

In [10]:
movies_with_genres.head()

,movieId,title,genres,IMAX,Western,Children,Drama,Adventure,Film-Noir,Thriller,...,Crime,Animation,Action,Horror,War,(no genres listed),Romance,Musical,Documentary,Sci-Fi
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#specify model parameters
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
#fit model to the data set
model_knn.fit(movie_wide)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
#Gets the top 10 nearest neighbours got the movie
def print_similar_movies(query_index) :
    #get the list of user ratings for a specific userId
    query_index_movie_ratings = movie_wide.loc[query_index,:].values.reshape(1,-1)
    #get the closest 10 movies and their distances from the movie specified
    distances,indices = model_knn.kneighbors(query_index_movie_ratings,n_neighbors = 11) 
    #write a loop that prints the similar movies for a specified movie.
    for i in range(0,len(distances.flatten())):
        #get the title of the random movie that was chosen
        get_movie = movie_list.loc[movie_list['movieId']==query_index]['title']
        #for the first movie in the list i.e closest print the title
        if i==0:
            print('Recommendations for {0}:\n'.format(get_movie))
        else :
            #get the indiciees for the closest movies
            indices_flat = indices.flatten()[i]
            #get the title of the movie
            get_movie = movie_list.loc[movie_list['movieId']==movie_wide.iloc[indices_flat,:].name]['title']
            #print the movie
            print('{0}: {1}, with distance of {2}:'.format(i,get_movie,distances.flatten()[i]))

In [14]:
print_similar_movies(1125)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recommendations for 1097    Return of the Pink Panther, The (1975)
Name: title, dtype: object:

1: 4378    Caddyshack II (1988)
Name: title, dtype: object, with distance of 0.6292718590715319:
2: 3967    Dog Run (1996)
Name: title, dtype: object, with distance of 0.6655161904537465:
3: 1245    Young Frankenstein (1974)
Name: title, dtype: object, with distance of 0.7183137062703213:
4: 4433    Salome's Last Dance (1988)
Name: title, dtype: object, with distance of 0.7394081508639004:
5: 2926    Re-Animator (1985)
Name: title, dtype: object, with distance of 0.7453332494633832:
6: 3643    Shaft (2000)
Name: title, dtype: object, with distance of 0.7459505703828824:
7: 360    Little Buddha (1993)
Name: title, dtype: object, with distance of 0.7484857814724895:
8: 4020    Jaws: The Revenge (1987)
Name: title, dtype: object, with distance of 0.7666891685552704:
9: 5562    Punch-Drunk Love (2002)
Name: title, dtype: object, with distance of 0.7711301367628625:
10: 3591    Class of Nuke 'Em 

In [15]:
print_similar_movies(1)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recommendations for 0    Toy Story (1995)
Name: title, dtype: object:

1: 341    Backbeat (1993)
Name: title, dtype: object, with distance of 0.4822207504730954:
2: 338    Baby-Sitters Club, The (1995)
Name: title, dtype: object, with distance of 0.511182036311638:
3: 463    Englishman Who Went Up a Hill But Came Down a ...
Name: title, dtype: object, with distance of 0.5118050001696173:
4: 538    So I Married an Axe Murderer (1993)
Name: title, dtype: object, with distance of 0.5259866830818456:
5: 443    Fearless (1993)
Name: title, dtype: object, with distance of 0.5442422496214109:
6: 351    Forrest Gump (1994)
Name: title, dtype: object, with distance of 0.5610339361142351:
7: Series([], Name: title, dtype: object), with distance of 0.5635264704525003:
8: 293    Panther (1995)
Name: title, dtype: object, with distance of 0.5764809422228946:
9: 539    Striking Distance (1993)
Name: title, dtype: object, with distance of 0.582777804572816:
10: 540    Super Mario Bros. (1993)
Name: t

In [16]:
print_similar_movies(6079)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recommendations for 5967    Five Days One Summer (1982)
Name: title, dtype: object:

1: 5698    8 Mile (2002)
Name: title, dtype: object, with distance of 0.6717868858846081:
2: 4833    Final Countdown, The (1980)
Name: title, dtype: object, with distance of 0.6849047398331823:
3: 5035    Bad News Bears in Breaking Training, The (1977)
Name: title, dtype: object, with distance of 0.6865776541512914:
4: 6200    Private Function, A (1984)
Name: title, dtype: object, with distance of 0.6981369839439786:
5: 5455    City by the Sea (2002)
Name: title, dtype: object, with distance of 0.6984552418399009:
6: 6072    Man Who Fell to Earth, The (1976)
Name: title, dtype: object, with distance of 0.7011916937146465:
7: 2874    Straight Story, The (1999)
Name: title, dtype: object, with distance of 0.7023124999238357:
8: 6156    Raising Victor Vargas (2002)
Name: title, dtype: object, with distance of 0.7082861644747294:
9: 5188    Sweetest Thing, The (2002)
Name: title, dtype: object, with distan

In [17]:
movies_with_genres.head()

,movieId,title,genres,IMAX,Western,Children,Drama,Adventure,Film-Noir,Thriller,...,Crime,Animation,Action,Horror,War,(no genres listed),Romance,Musical,Documentary,Sci-Fi
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Getting the movies list with only genres like Musical and other such columns
movie_content_df_temp = movies_with_genres.copy()
movie_content_df_temp.set_index('movieId')
movie_content_df = movie_content_df_temp.drop(columns = ['movieId','title','genres'])
#movie_content_df = movie_content_df.as_matrix()
movie_content_df

,War,Fantasy,IMAX,Mystery,Action,Western,(no genres listed),Sci-Fi,Thriller,Documentary,Crime,Musical,Horror,Comedy,Drama,Film-Noir,Romance,Children,Animation,Adventure
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
62419,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62420,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
62421,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(movie_content_df,movie_content_df)